# 데이터 분할을 위한 폴더 생성

In [2]:
import os
import shutil

original_dataset_dir = '/Users/b._.chan/Documents/TABA_Project/Leaf_practice/dataset' # 원본 데이터셋이 위치한 경로 지정
classes_list = [cls for cls in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, cls))] 
# os.listdir() : 해당 경로 하위에 있는 모든 폴더의 목록을 가져오는 메소드

base_dir = './splitted' # 나눈 데이터를 저장할 폴더 생성
os.mkdir(base_dir)

# 분리 후에 각 데이터를 저장할 하위 폴더 train, val, test create
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# train, validation, test 폴더 하위에 각각 클래스 목록 폴더 생성
for clss in classes_list:
    os.mkdir(os.path.join(train_dir, clss))
    os.mkdir(os.path.join(validation_dir, clss))    
    os.mkdir(os.path.join(test_dir, clss))
    

# 데이터 분할과 클래스별 데이터 수 확인

In [3]:
import math

# for문을 통해 모든 클래스에 대한 작업 반복
for clss in classes_list:
    path = os.path.join(original_dataset_dir, clss)
    fnames = [fname for fname in os.listdir(path) if os.path.isfile(os.path.join(path, fname)) and not fname.startswith('.')] 
    # path 위치에 존재하는 모든 이미지 파일의 목록을 변수 fnames에 저장
    
    # Train, Validation, Test data의 비율을 지정. 이 프로젝트에서는 6:2:2 비율 지정
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    
    train_fnames = fnames[:train_size]
    print('Train size(', clss, '): ', len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, clss), fname)
        shutil.copyfile(src, dst)
    
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print('Validation size(', clss, '): ', len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, clss), fname)
        shutil.copyfile(src, dst)
    
    test_fnames = fnames[(train_size + validation_size):
        (validation_size + train_size + test_size)]
    
    print('Test size(', clss, '): ', len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, clss), fname)
        shutil.copyfile(src, dst)

Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Potato___Early_blight ):  600
Validation size( Potato___Early_blight ):  200
Test size( Potato___Early_blight ):  200
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Tomato___Target_Spot ):  842
Validation size( Tomato___Target_Spot ):  280
Test size( Tomato___Target_Spot ):  280
Train size( Peach___healthy ):  216
Validation size( Peach___healthy ):  72
Test size( Peach___healthy ):  72
Train size( Potato___Late_blight ):  600
Validation size( Potato___Late_blight ):  200
Test size( Potato___Late_blight ):  200
Train size( Tomato___Late_blight ):  1145
Validation size( Tomato___Late_blight ):  381
Test size( Tomato___Late_blight ):  381

## 베이스라인 모델 학습을 위한 준비

In [4]:
import torch

USE_MPS = torch.mps.is_available() # 현재 사용중인 환경에서 GPU를 사용할 수 있는지를 확인하는 Method (True/False)
DEVICE = torch.device('mps' if USE_MPS else "cpu") # 사용하는 장비가 무엇인지를 저장하는 변수 (True : mps, False : cpu)

BATCH_SIZE = 256 
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

# Compose : 이미지 데이터의 전처리, Augmentation(좌우 반전, 밝기 조절, 이미지 임의 확대) 등의 과정에서 사용되는 method,, 
# Augmentation 을 통해 이미지에 노이즈를 주어 더욱 강건한 모델을 만들 수 있음.
transform_base = transforms.Compose([transforms.Resize((64, 64)), 
                                     transforms.ToTensor()])

# ImageFolder method : 데이터셋을 불러오는 메소드 (이미지 데이터는 하나의 클래스가 하나의 폴더에 대응됨)
train_dataset = ImageFolder(root='./splitted/train',
                            transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val',
                          transform=transform_base)

# DataLoader는 불러온 이미지 데이터를 주어진 조건에 따라 Mini-batch 단위로 분리하는 역할 수행
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True,
                                           num_workers=4)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=4)

## Baseline model 설계

In [5]:
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module): # 딥러닝 모델과 관련된 기본적인 함수를 포함하는 nn.Module 클래스를 상속하여 사용
    def __init__(self): # 클래스 내부의 __init__ 함수에서는 모델에서 사용할 모든 Layer 정의
        super(Net, self).__init__() # nn.Module 내에 있는 메서드를 상속받아 사용
        
        # 첫 번째 2d Convolutional Layer 정의 
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) 
        self.pool = nn.MaxPool2d(2, 2) # Kernel size =2, Stride = 2
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        
        self.fc1 = nn.Linear(4096, 512) # Flatten 이후에 사용될 첫 번째 Fully-connected Layer 정의
        self.fc2 = nn.Linear(512, 33)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training) # training = self.training : 학습 모드일 때와 검증 모드일 때 각각 다르게 적용되기 위해 존재
        # 학습 과정에서는 일부 노드를 랜덤하게 제외시키지만, 평가 과정에서는 모든 노드를 사용
        x = self.conv2(x) # Convolution 연산을 진행한 후 Feature Map 생성
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training) 
        
        x = x.view(-1, 4096) # Flatten 연산 -> 1차원으로 펼침
        x = self.fc1(x) # Flatten 된 1차원 Tensor를 fc1에 통과
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x) # 마지막 Layer로 클래스의 개수에 해당하는 33개의 출력값을 가짐.
        
        output = F.log_softmax(x, dim=1) # 마지막 출력값에 softmax() 함수를 적용하여 데이터가 각 클래스에 속할 확률을 출력
        
        return output
    
model_base = Net().to(DEVICE) # 정의한 CNN 모델 Net()의 새로운 객체 생성 후 모델을 현재 사용중인 장비에 할당
optimizer = optim.Adam(model_base.parameters(), lr=0.001)
        
        

## 모델 학습을 위한 함수

In [6]:
def train(model, train_loader, optimizer):
    model.train() # 입력받는 모델을 학습 모드로 설정
    # train_loader 에는 (data, target) 형태가 Mini-batch 단위로 묶여 있음
    # train_loader에 enumerate() 함수를 적용했기 때문에 batch_idx, (data, target) 형태로 반복 가능한 객체가 생성되어
    # for문 생성
    for batch_idx, (data, target) in enumerate(train_loader): 
        data, target = data.to(DEVICE), target.to(DEVICE) # data와 target 변수를 사용 중인 장비에 할당
        optimizer.zero_grad() # 이전 Batch의 Gradient 값이 optimizer에 저장되어 있으므로 optimizer 초기화
        output = model(data) # 데이터를 모델에 입력하여 Output 값 계산
        loss = F.cross_entropy(output, target) # 모델에서 계산한 Output 값은 예측값과 Target 값 사이의 Loss 계산.. 분류 문제에 적합한 Cross Entropy Loss 사용
        loss.backward() # 역전파 알고리즘 -> Loss 값을 바탕으로 Back Propagation을 통해 계산한 Gradient 값을 각 Parameter에 할당
        optimizer.step() # Parameter를 Gradient 값을 이용해 업데이트

## 모델 평가를 위한 함수

In [7]:
def evaluate(model, test_loader):
    model.eval() # 입력받는 모델을 평가 모드로 설정
    test_loss = 0 # Mini-batch 별로 Loss를 합산해서 저장할 변수
    correct = 0 # 올바르게 예측한 데이터 수를 세는 변수
    
    with torch.no_grad(): # 모델을 평가하는 단계에서는 모델의 Parameter를 업데이트하지 않아야 함.
        # torch.no_grad() : 해당 부분을 실행하는 동안 모델의 Parameter 업데이트 중단
        for data, target in test_loader: # for문을 통해 데이터와 대응하는 Label 값에 접근
            data, target = data.to(DEVICE), target.to(DEVICE) 
            output = model(data) # 데이터를 모델에 입력하여 output 값 계산
            
            # 모델에서 계산한 output 값인 예측값과 target 값 사이의 Loss 계산
            # 성능 평가 과정에서도 Cross Entropy Loss 함수 사용
            test_loss = F.cross_entropy(output,
                                        target, reduction="sum").item()
            
            # 모델에 입력된 Test Data가 33개의 클래스에 속할 확률값이 Output으로 출력되는데, 가장 높은 값을 가진 인덱스를 예측값을 저장
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            # target.view_as(pred)를 통해 target Tensor 구조를 pred Tensor와 같은 모양으로 정렬
            # view_as() method : 적용 대상 Tensor를 메서드에 입력되는 Tensor 모양대로 재정렬하는 함수
            # eq() method : 객체 간의 비교 연산자로 일치하면 1, 일치하지 않으면 0 반환
    test_loss /= len(test_loader.dataset) # 모든 Mini-batch에서 합한 Loss 값을 Batch 수로 나누어 미니 배치마다 계산된 Loss 값 평균
    test_accuracy = 100. * correct / len(test_loader.dataset) # 모든 Mini-batch에서 합한 정확도 값을 Batch 수로 나누어 미니 배치마다 계산된 정확도 평균
    return test_loss, test_accuracy

## 모델 학습 실행

In [8]:
import time
import copy

def train_baseline(model, train_loader, val_loader,
                   optimizer, num_epochs=30):
    best_acc = 0.0 # 정확도가 가장 높은 모델의 정확도를 저장하는 변수 선언
    best_model_wts = copy.deepcopy(model.state_dict()) # 정확도가 가장 높은 모델을 저장할 변수 선언
    
    for epoch in range(1, num_epochs + 1): 
        since = time.time()     # 한 Epoch 당 소요되는 시간을 측정하기 위해 해당 Epoch을 시작할 때의 시각 저장
        train(model, train_loader, optimizer) # 모델 학습
        train_loss, train_acc = evaluate(model, train_loader) # 학습 Loss, 정확도 계산
        val_loss, val_acc = evaluate(model, val_loader) # 검증 Loss, 정확도 계산
        
        if val_acc > best_acc: # 현재 Epoch의 검증 정확도가 최고 정확도보다 높다면 best_acc를 현재 Epoch의 검증 정확도로 업데이트
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict()) # 해당 Epoch의 모델을 best_model_wts에 저장
            
        time_elapsed = time.time() - since # 한 Epoch 당 소요된 시간 계산
        print(f'--------------- epoch {epoch} -------------------')
        
        print('train_loss : {:.4f}, Accuracy : {:.2f}%'.format(train_loss, train_acc)) # 학습 Loss, 정확도 출력
        print('val_loss : {:.4f}, val_acc : {:.2f}%'.format(val_loss, val_acc)) # 검증 Loss, 정확도 출력
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # 한 Epoch 당 소요된 시간 출력
    
    model.load_state_dict(best_model_wts) # 최종적으로 정확도가 가장 높은 모델을 불러온 뒤, 반환
    return model

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH) # Baseline 모델 학습

torch.save(base, 'baseline.pt') # 학습이 완료된 모델 저장

--------------- epoch 1 -------------------
train_loss : 0.0106, Accuracy : 57.08%
val_loss : 0.0097, val_acc : 56.83%
Completed in 1m 30s
--------------- epoch 2 -------------------
train_loss : 0.0072, Accuracy : 68.44%
val_loss : 0.0057, val_acc : 67.34%
Completed in 1m 30s
--------------- epoch 3 -------------------
train_loss : 0.0068, Accuracy : 73.25%
val_loss : 0.0060, val_acc : 71.86%
Completed in 1m 29s
--------------- epoch 4 -------------------
train_loss : 0.0036, Accuracy : 83.18%
val_loss : 0.0045, val_acc : 81.22%
Completed in 1m 28s
--------------- epoch 5 -------------------
train_loss : 0.0044, Accuracy : 84.91%
val_loss : 0.0052, val_acc : 82.73%
Completed in 1m 29s
--------------- epoch 6 -------------------
train_loss : 0.0034, Accuracy : 84.79%
val_loss : 0.0037, val_acc : 81.99%
Completed in 1m 29s
--------------- epoch 7 -------------------
train_loss : 0.0027, Accuracy : 88.96%
val_loss : 0.0022, val_acc : 86.11%
Completed in 1m 29s
--------------- epoch 8 ---

# Transfer Learning을 위한 준비

In [9]:
data_transforms = {
    'train': transforms.Compose([ # transforms.Compose() : 이미지 데이터의 전처리, Augmentation 등의 과정에서 사용
        transforms.Resize([64, 64]), # transforms.Resize([64, 64]) : 이미지의 크기를 64 * 64로 조정
        # Image Augmentation,, 이미지를 무작위로 좌우 반전.. 괄호 안에 Parameter p를 입력하여 반전되는 이미지의 비율을 설정할 수 있음 (Default : 0.5)
        transforms.RandomHorizontalFlip(), 
        transforms.RandomVerticalFlip(), # 이미지를 무작위로 상하 반전하는 것 (Default : 0.5)
        transforms.RandomCrop(52), # 이미지의 일부를 랜덤으로 잘라 내어  52*52 사이즈로 변경
        transforms.ToTensor(), # 이미지를 Tensor 형태로 변환하고 모든 숫자를 0에서 1 사이로 변경
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]) # 이미지가 Tensor 형태로 전환된 이후에 정규화 시행 -> 정규화 위해서는 평균값과 표준편차 값 필요
        # 첫 번째 대괄호[] 는 Red, Green, Blue 채널 값에서 정규화를 적용할 평균값을 의미
        # 두 번째 대괄호[] 는 Red, Green, Blue 채널 값에서 정규화를 적용할 표준편차 값을 의미
        # 이 데이터들은 Pre-trained Model 학습에 사용된 ImageNet 데이터 값
        # 입력 데이터 정규화는 모델을 최적화하고, Local Mininum에 빠지는 것을 방지하는 데 도움이 됨.
    ]),
    
    # 검증에 사용되는 데이터에는 학습에 적용했던 전처리에서 Augmentation에 해당하는 부분을 제외한 나머지 부분을 동일하게 적용
    'val': transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

data_dir = './splitted' # 학습 데이터와 검증 데이터를 불러올 폴더 경로 설정

# ImageFolder 메서드는 데이터셋을 불러오는 Method
# transform 옵션에는 데이터를 불러온 후 전처리 또는 Augmentation 방법 지정 -> transform_base로 옵션 지정
# 훈련, 검증 과정에서 각각의 과정에 맞는 데이터를 편리하게 불러오고자 Dictionary 형태로 구성
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x),
                                 transform=data_transforms[x]) for x in ['train', 'val']}

# DataLoader는 불러온 이미지 데이터를 주어진 조건에 따라 Mini-batch 단위로 분리하는 역할
# image_dataset을 이용하여 생성하고, Shuffle=True로 설정하여 Label 정보의 순서를 기억하는 것을 방지
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']} # 학습 데이터와 검증 데이터의 총 개수를 각각 저장

class_names = image_datasets['train'].classes # 33개의 클래스 이름 목록을 저장

## Pre-trained Model load

In [10]:
from torchvision import models

resnet = models.resnet50(pretrained=True) # pretrained = True : 미리 학습된 모델의 Parameter 값을 그대로 가져옴 (False : 모델의 구조만 -> Parameter : 랜덤)
num_ftrs = resnet.fc.in_features # 이 프로젝트에서는 33개의 클래스로 분류해야 하기 때문에 마지막 Layer 출력 채널 수는 33개이어야 함.
# ResNet50 모델은 다른 주제를 위해 설계되었기에 마지막 Layer의 출력 채널 수가 33개가 아님
# 이 프로젝트의 주제에 맞추고자 모델의 마지막 FCL 대신 출력 채널의 수가 33개인 새로운 Layer 추가
# ResNet50에서 마지막 Layer의 입력 채널 수를 저장,, in_features 는 해당 Layer의 입력 채널 수를 의미

resnet.fc = nn.Linear(num_ftrs, 33) # 불러온 모델의 마지막 FCL을 새로운 Layer로 교체,,  입력 채널의 수는 기존 Layer와 동일하고 출력 채널 수는 목적에 맞게 설정
resnet = resnet.to(DEVICE) # 모델을 현재 사용 중인 장비에 할당

criterion = nn.CrossEntropyLoss() # 모델을 학습할 때 사용하는 Loss 함수를 지정하는 변수

# 이 모델에서는 설정한 일부 Layer의 Parameter만을 업데이트 해야 해서 filter() method와 lambda 표현식을 사용하여
# requires_grad = True로 설정된 Layer의 Parameter에만 적용
optimizer_ft = optim.Adam(filter(lambda p : p.requires_grad,
                                 resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler

# StepLR() method는 Epoch에 따라 Learning Rate를 변경하는 역할을 함
# step_size = 7, gamma = 0.1로 설정하면 7 Epoch 마다 0.1씩 곱해 Learning Rate를 감소시킨다는 의미
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,
                                       step_size=6, gamma=0.1)

/opt/homebrew/Caskroom/miniconda/base/envs/TABA/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/TABA/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Pre-Trained Model 일부 Layer Freeze

In [11]:
ct = 0 # 해당 Layer가 몇 번째 Layer인지 나타내는 변수
for child in resnet.children(): # children method : 모델의 자식 모듈을 반복 가능한 객체로 반환하는 method
    # resnet.children() method : 생성한 resnet 모델의 모든 Layer 정보를 담고 있음.
    ct += 1 # for 문을 한 번 반복한 후에는 다음 Layer를 지칭하도록 변수 ct 값을 1만큼 증가
    if ct < 6: # 미리 학습된 모델의 일부 Layer만 업데이트 하도록 설정하는 방법은 Parameter를 업데이트하지 않을 상위 Layer들의 requires_grad 값을 False로 지정
        for param in child.parameters(): # ResNet50에 존재하는 10개의 Layer 중에서 1번부터 5번 Layer의 Parameter는 업데이트 되지 않도록 고정하고,
            # 6번부터 10번 Layer의 Parameter는 학습 과정에서 업데이트하도록 설정
            # child.parameters() : 각 Layer의 Parameter Tensor를 의미.. requries_grad : True
            # requires_grad = False : Parameter가 업데이트되지 않도록 설정한다는 의미,, Layer의 번호가 6보다 작을 경우에는 Parameter가 업데이트 되지 않도록 설정
            param.requires_grad = False

## Transfer Learning 모델 학습과 검증을 위한 함수

In [14]:
import copy
import time
def train_resnet(model, criterion, optimizer, schedular, num_epochs=25):
    
    best_model_wts = copy.deepcopy(model.state_dict()) # 정확도가 가장 높은 모델을 저장할 변수
    
    best_acc = 0.0 # 정확도가 가장 높은 모델의 정확도를 저장하는 변수 선언

    for epoch in range(num_epochs):
        print('----------------- epoch {} -------------'.format(epoch + 1)) # 현재 진행중은 Epoch
        since = time.time() # 한 Epoch 당 소요되는 시간을 측정하기 위해 시작 시각 저장
        
        for phase in ['train', 'val']: # 한 Epoch은 각각 학습 단계와 검증 단계를 가지고, 한 Epoch 마다 학습 모드와 검증 모드를 각각 실행
            if phase == 'train': # 상황에 적합하게 모델을 학습 모드로 설정
                model.train()
            else:
                model.eval() # 상황에 적합하게 모델을 검증 모드로 설정
            
            
            running_loss = 0.0 # 모든 데이터의 Loss를 합산해서 저장할 변수
            running_corrects = 0 # 올바르게 예측한 경우의 수를 세는 변수
            
            for inputs, labels in dataloaders[phase]: # 모델의 현재 모드에 해당하는 DataLoader에서 데이터 입력
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE) # 데이터와 해당하는 Label 값을 현재 사용중인 장비에 할당
                
                optimizer.zero_grad() # Gradient 초기화
                
                with torch.set_grad_enabled(phase == 'train'): # 학습 단게에서만 모델 Gradient를 업데이트하고, 검증 단게에서는
                    # 업데이트 하지 않아야 하므로 set_grad_enabled() method를 사용하여 phase가 train일 경우에만 업데이트하도록 설정
                    outputs = model(inputs) # 데이터를 모델에 입력하여 Output 값 계산
                    _, preds = torch.max(outputs, 1) # 모델에 입력된 test 데이터가 33개의 클래스에 속할 각각의 확률값이
                    # Output으로 출력되고 그 중 가장 높은 값을 가진 인덱스를 예측값으로 저장
                    loss = criterion(outputs, labels) 
                    
                    # 모델이 현재 학습 모드인 경우 앞에서 계산한 Loss 값을 바탕으로 Back Propagation을 통해 계산한 
                    # Gradient 값을 각 Parameter에 할당하고, 모델의 Parameter를 Update
                    if phase == 'train': 
                        loss.backward()
                        optimizer.step()
                
                # 모든 데이터 Loss를 합산해서 저장하고자 하나의 Mini-batch에 대해 계산된 Loss 값에 데이터의 수를 곱해 합산
                # 이 때 inputs.size(0)은 DataLoader에서 전달되는 Mini-batch의 데이터 수를 의미하는 것으로, Batch_size        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data) # 앞에서 모델을 통해 예측한 값과 Target이 같으면 running_corrects를 1만큼 증가
            if phase == 'train': # 7 Epoch 마다 Learning Rate를 다르게 조정하는 Scheduler와 관련된 부분,, 한 Epoch당 1번 모델이 현재 학습 단계일 경우에만 실행
                schedular.step()
                # Scheduler에 의해 Learning Rate가 조정되는 것을 직접 확인하기 위한 부분
                # optimizer_ft.param_groups의 원소는 학습 과정에서의 Parameter를 저장하고 있는 dictionary
                # 이 중 Learning Rate에 해당하는 Key인 "lr"를 이용하여 각 Epoch의 Learning Rate를 불러옴
                l_r = [x['lr'] for x in optimizer_ft.param_groups] 
                print('learning rate : ' , l_r)
        
            # 해당 Epoch의 Loss를 계산하기 위해 running_loss를 미리 계산해 둔 dataset_size로 나눔
            epoch_loss = running_loss / dataset_sizes[phase]
            # 해당 Epoch의 정확도를 계산하기 위해 running_corrects를 미리 계산해 둔 dataset_size로 나눔
            epoch_acc = running_corrects.float() / dataset_sizes[phase]
            
            # 해당 Epoch과 현재 모델의 단계, Loss 값, 정확도를 출력
            print('{} loss : {:.4f} Acc : {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # 검증 단게에서 현재 Epoch의 정확도가 최고 정확도보다 높다면 
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc # best_acc를 현재 Epoch의 정확도로 업데이트
                best_model_wts = copy.deepcopy(model.state_dict()) # 해당 Epoch의 모델을 best_model_wts에 저장
                
        time_elasped = time.time() - since
        
        print('completed in {:.0f}m {:0f}s'.format(time_elasped // 60, time_elasped % 60))
    print('Best val Acc : {:.4f}'.format(best_acc))

    model.load_state_dict(best_model_wts) # 정확도가 가장 높은 모델을 불러온 후 반환
    
    return model

# 모델 학습 실행

In [15]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft,
                              exp_lr_scheduler, num_epochs=EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

----------------- epoch 1 -------------
learning rate :  [0.001]
train loss : 0.2196 Acc : 0.9279
val loss : 0.2219 Acc : 0.9313
completed in 1m 52.473299s
----------------- epoch 2 -------------
learning rate :  [0.001]
train loss : 0.1728 Acc : 0.9430
val loss : 0.1982 Acc : 0.9345
completed in 3m 1.150282s
----------------- epoch 3 -------------
learning rate :  [0.001]
train loss : 0.1399 Acc : 0.9524
val loss : 0.1707 Acc : 0.9456
completed in 3m 6.154328s
----------------- epoch 4 -------------
learning rate :  [0.001]
train loss : 0.1099 Acc : 0.9633
val loss : 0.1364 Acc : 0.9569
completed in 2m 34.458076s
----------------- epoch 5 -------------
learning rate :  [0.0001]
train loss : 0.0961 Acc : 0.9685
val loss : 0.1264 Acc : 0.9591
completed in 2m 39.678178s
----------------- epoch 6 -------------
learning rate :  [0.0001]
train loss : 0.0494 Acc : 0.9840
val loss : 0.0524 Acc : 0.9834
completed in 2m 56.966598s
----------------- epoch 7 -------------
learning rate :  [0.0001

# 모델 평가

## 베이스라인 모델 평가를 위한 전처리

In [16]:
# 베이스라인 모델의 성능 평가를 위해 사용할 테스트 데이터의 DataLoader 생성
# 모델을 학습시킬 때 사용한 학습, 검증 데이터와 동일한 방법으로 전처리 수행, 배치 사이즈 동일
transform_base = transforms.Compose([transforms.Resize([64, 64]),
                                     transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',
                        transform=transform_base)

test_loader_base = torch.utils.data.DataLoader(test_base,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=4)

## Transfer Learning Model evaluation을 위한 Preprocessing

In [17]:
# Transfer Learning model 성능 평가를 위해 사용할 테스트 데이터의 DataLoader 생성
# Transfer Learning model의 경우에도 마찬가지로 모델을 학습시킬 때 사용한 학습, 검증 데이터와 동일한 방법으로 전처리 수행
# 배치 사이즈도 동일하게 설정

transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_resNet = ImageFolder(root='./splitted/test',
                          transform=transform_resNet)

test_loader_resNet = torch.utils.data.DataLoader(dataset=test_resNet,
                                                 batch_size=BATCH_SIZE,
                                                 shuffle=True,
                                                 num_workers=4)

## 베이스라인 모델 성능 평가

In [19]:
# 저장했던 베이스라인 모델 불러옴
baseline = torch.load('baseline.pt')
baseline.eval() # 모델을 평가 모드로 설정
# 앞서 정의한 evaluate 함수를 이용하여 테스트 데이터에 대한 정확도 측정
test_loss, test_accuracy = evaluate(baseline, test_loader=test_loader_base)

print('baseline test acc: ', test_accuracy) # 평가 정확도 출력



/var/folders/t6/dngzdyds4h36v9fcy0qzhlj40000gn/T/ipykernel_81044/1274393065.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline = torch.load('baseline.pt')


baseline test acc:  94.07935911878833


## Transfer Learning 모델 성능 평가

In [20]:
# 저장된 Transfer Learning model 불러오기
resnet50 = torch.load('resnet50.pt')
resnet50.eval() # 모델을 평가 모드로 설정
# 앞서 정의한 evaluate 함수를 이용하여 테스트 데이터에 대한 정확도 측정
test_loss, test_accuracy = evaluate(model=resnet50, test_loader=test_loader_resNet)

# 평가 정확도 출력
print(f"ResNet test acc : {test_accuracy}")

/var/folders/t6/dngzdyds4h36v9fcy0qzhlj40000gn/T/ipykernel_81044/89219646.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet50 = torch.load('resnet50.pt')


ResNet test acc : 98.86093378395293
